In [594]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [595]:
results = pd.read_csv('mpc_results.csv')
results

,Unnamed: 0,battery_bid_fcas_power,battery_bid_fcas_price,battery_bid_fcas_type,battery_bid_energy_power_gen,battery_bid_energy_power_load,battery_bid_energy_price,battery_bid_energy_type,fcas_clearing_price,energy_price,raise_demand,raise_price,soe,timestamp
0,0,4.605535e+01,41.00,gen,0.000000e+00,3.000000e+02,54.90,load,41.00,54.90,41.00,27.81000,0.000000,2018-06-01 04:30:00
1,1,6.328406e+01,41.00,gen,0.000000e+00,3.000000e+02,56.19,load,41.00,56.19,41.00,28.50000,292.001095,2018-06-01 05:00:00
2,2,1.100044e+02,41.00,gen,0.000000e+00,3.000000e+02,58.73,load,41.00,58.73,41.00,24.97000,584.002191,2018-06-01 05:30:00
3,3,2.000000e+02,41.67,gen,0.000000e+00,2.529967e+02,62.98,load,41.67,62.98,41.67,20.59000,876.003285,2018-06-01 06:00:00
4,4,2.000000e+02,47.67,gen,0.000000e+00,1.000000e+02,74.24,load,47.67,74.24,47.67,18.24996,1029.000000,2018-06-01 06:30:00
5,5,2.000000e+02,75.59,gen,-3.000000e+02,0.000000e+00,229.57,gen,75.59,229.57,75.59,30.00000,1029.000000,2018-06-01 07:00:00
6,6,2.000000e+02,62.14,gen,-2.037028e+02,0.000000e+00,104.33,gen,62.14,104.33,62.14,42.27879,703.702762,2018-06-01 07:30:00
7,7,2.000000e+02,81.00,gen,-3.000000e+02,0.000000e+00,112.42,gen,81.00,112.42,81.00,35.01569,481.432226,2018-06-01 08:00:00
8,8,2.000000e+02,81.00,gen,-5.343227e+01,0.000000e+00,103.99,gen,81.00,103.99,81.00,30.00000,153.432268,2018-06-01 08:30:00
9,9,2.000000e+02,78.92,gen,0.000000e+00,3.000000e+02,94.00,load,78.92,94.00,78.92,20.00000,72.000042,2018-06-01 09:00:00


In [596]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['soe'], name='soe'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [597]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='raise power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_price'], name='raise price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [598]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_load'], name='energy power load'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_gen'], name='energy power gen'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [599]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['battery_bid_fcas_price'], name='fcas bid'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='fcas power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['fcas_clearing_price'], name='fcas cleared'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_demand'], name='fcas demand'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Fcas Price ($/MW)", secondary_y=False)
fig.update_yaxes(title_text="Fcas Demand (MW)", secondary_y=True)

fig.show()